# 01a – Bugfix-Validierung: AVSR Baseline nach Segmentation-Fix


## Hintergrund: Der Bug in `segmentation.py`

In `mcorec_baseline/src/talking_detector/segmentation.py` wurde ein Copy-Paste-Fehler
in der Parameterkonvertierung entdeckt. Die `min_duration_off`-Variable las fälschlicherweise
den Defaultwert von `min_duration_on`:

```python
# FEHLERHAFT (vor dem Fix):
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_on"]) * 25)  # ← falscher Key!

# KORREKT (nach dem Fix):
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_off"]) * 25)
```

Da `min_duration_on` und `min_duration_off` unterschiedliche Defaultwerte haben, führte
der Bug dazu, dass die Segmentierungsschwelle für Sprechpausen falsch berechnet wurde.
Dieses Notebook reevaluiert alle vier Baseline-Modelle mit dem korrigierten Code,
um die Metriken mit denen ohne Bugfix vergleichen zu können.

Die folgenden Experimente in den Notebooks 2 bis 2j wurden mit der ursprünglichen nicht angepassten segmentation.py durchgeführt.

> **Output-Ordner:** Alle Ergebnisse landen in `output_bugfix_*`-Verzeichnissen,
> damit sie nicht mit den Pre-Bugfix-Ergebnissen aus `01_avsr_baseline_eval.ipynb` vermischt werden.

## 1 – GPU-Verfügbarkeit prüfen

Vor Ausführung des Codes wird prüft, welche GPUs frei sind, damit es nicht zu einem Out of Memory Fehler kommt.

In [1]:
!nvidia-smi

Fri Jan 30 21:59:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   31C    P0             79W /  500W |    7093MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2 – GPU-Auswahl

In [16]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Anpassen je nach Verfügbarkeit

## 3 – Konfiguration

In [11]:
# Absoluter Pfad zum geklonten mcorec_baseline-Repository (je nach Nutzer:in anpassen)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"

session = "*" # alle Sessions oder einzelne session_132

# Pfad-Pattern für die Session-Verzeichnisse relativ zum Repo-Root
session_dir = f"data-bin/dev/{session}"

## 4 – Setup: Arbeitsverzeichnis & Imports

In [17]:
import os, sys

# Arbeitsverzeichnis auf Repo-Root setzen, damit alle relativen Pfade korrekt aufgelöst werden
os.chdir(project_baseline_path)

# Zur Verifikation ausgeben
print("CWD:", os.getcwd())

CWD: /home/josch080/Projektgruppe/mcorec_baseline


In [5]:
from script import inference

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 5 – Modell-Definitionen

Gleiche vier Modelle wie in `01_avsr_baseline_eval.ipynb`, aber mit `output_bugfix_`-Prefix
im Output-Verzeichnisnamen, um Pre- und Post-Bugfix-Ergebnisse klar zu trennen.

In [7]:
MODELS = {

     #BL1: AV-HuBERT "Cocktail" (CTC/Attention)
    "cocktail_base": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail",
        "out": "output_bugfix_avsr_cocktail"
    },

     #BL2: MuAViC-EN + AV-HuBERT Transformer-Decoder
    "muavic_en": {
        "model_type": "muavic_en",
        "chkpt": "nguyenvulebinh/AV-HuBERT-MuAViC-en",
        "out": "output_bugfix_muavic_en"
    },
    
     #BL3: Auto-AVSR mit Conformer (CTC/Attention)
    "auto": {
        "model_type": "auto_avsr",
        "chkpt": "model-bin/auto_avsr/avsr_trlrwlrs2lrs3vox2avsp_base.pth",
        "out": "output_bugfix_auto_avsr"
    },

     #BL4: AV-HuBERT "Cocktail" (MCoRec-feingetuned)
    # Dasselbe Grundmodell wie BL1 aber auf MCoRec-Trainingsdaten feinjustiert
    "cocktail_fine": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
        "out": "output_bugfix_avsr_cocktail_finetuned"
    },
}    

## 6 – Inferenz

Die Inferenz wird für alle vier Modelle sequenziell ausgeführt.
Parameter: `beam_size=3`, `max_length=15` (Baseline-Default).

`inference.main()` ist auskommentiert, da die Ergebnisse bereits vorliegen. Für einen erneuten Durchlauf die Zeile einkommentieren.

In [7]:
# Alle vier Modelle nacheinander laufen lassen

for model_key, m in MODELS.items():
    print(f"\n=========================")
    print (f"Starte Inference für Modell: {model_key}")
    print (f"model_type = {m['model_type']}")
    print (f"checkpoint_path = {m['chkpt']}")
    print (f"output_dir_name = {m['out']}")
    print (f"session_dir = {session_dir}")

    sys.argv = [
    "notebook",
    "--model_type", m["model_type"],
    "--session_dir", session_dir,
    "--checkpoint_path", m["chkpt"],
    "--beam_size", "3",
    "--max_length", "15", 
    "--output_dir_name", m["out"],

    ]
    #inference.main()


Starte Inference für Modell: cocktail_base
model_type = avsr_cocktail
checkpoint_path = model-bin/avsr_cocktail
output_dir_name = output_bugfix_avsr_cocktail
session_dir = data-bin/dev_without_central_videos/dev/*
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail
avsr_cocktail model loaded successfully!
Inferring 25 sessions using avsr_cocktail model
Processing session session_44


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [01:07<05:39, 67.86s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 28/28 [00:30<00:00,  1.10s/it]

Processing speakers:  20%|██        | 1/5 [00:52<03:28, 52.16s/it]


Processing speakers:  40%|████      | 2/5 [01:47<02:42, 54.08s/it]


Processing speakers:  60%|██████    | 3/5 [02:35<01:42, 51.35s/it]


Processing speakers:  80%|████████  | 4/5 [02:50<00:36, 36.93s/it]


Processing speakers: 100%|██████████| 5/5 [04:16<00:00, 51.36s/it]


## 7 – Evaluation

`evaluate.py` mit Prefix `output_` gestartet – deckt alle `output_bugfix_*`-Verzeichnisse ab.
Log wird in `eval_log_bugfix_baseline.txt` gesichert.

In [12]:
import subprocess, sys, re, pandas as pd

eval_session_dir = session_dir   
eval_output_prefix = "output_"

# evaluate.py als Subprocess starten (sys.executable = gleicher Interpreter wie im Notebook)
cmd = [
    sys.executable,
    "script/evaluate.py",
    "--session_dir", eval_session_dir,
    "--output_dir_name", eval_output_prefix,
    "--label_dir_name", "labels",
]

# capture_output=True: stdout/stderr werden abgefangen statt direkt ausgegeben
result = subprocess.run(cmd, capture_output=True, text=True)

# Ausgabe trotzdem im Notebook sehen:
print(result.stdout)

# Ausgabe auch in eine Datei schreiben
with open("eval_log_bugfix_baseline.txt", "w") as f:
    f.write(result.stdout)


Evaluating 25 sessions

=== Evaluating session session_44 ===

--- Evaluating output dir: output_auto_avsr ---
Conversation clustering F1 score: 0.6
Speaker to WER: {'spk_0': 0.764, 'spk_1': 0.8032, 'spk_2': 0.8927, 'spk_3': 0.8484, 'spk_4': 0.8789, 'spk_5': 0.8425}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 0.5, 'spk_3': 0.5, 'spk_4': 0.5, 'spk_5': 0.5}
Joint ASR-Clustering Error Rate: {'spk_0': 0.382, 'spk_1': 0.4016, 'spk_2': 0.69635, 'spk_3': 0.6742, 'spk_4': 0.68945, 'spk_5': 0.67125}

--- Evaluating output dir: output_avsr_cocktail ---
Conversation clustering F1 score: 0.6
Speaker to WER: {'spk_0': 0.3618, 'spk_1': 0.6739, 'spk_2': 0.4554, 'spk_3': 0.5275, 'spk_4': 0.6101, 'spk_5': 0.6678}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 0.5, 'spk_3': 0.5, 'spk_4': 0.5, 'spk_5': 0.5}
Joint ASR-Clustering Error Rate: {'spk_0': 0.1809, 'spk_1': 0.33695, 'spk_2': 0.4777, 'spk_3': 0.5137499999999999, 'spk_4': 0.55505, 'spk_5': 0.5839}

--- Ev

## 8 – Ergebnisse parsen

Im Unterschied zu `01_avsr_baseline_eval.ipynb` wird hier gezielt **ein** Experiment
per `pattern.search()` extrahiert statt alle per `finditer()`.
Das reicht, weil der Fokus dieses Notebooks auf BL4 (dem besten Modell) liegt.

In [20]:
import re
import pandas as pd

# Ziel-Experiment: das post-Bugfix-Ergebnis von BL4 (bestes Modell)
# Hier kann auch ein anderer output_bugfix_*-Name eingesetzt werden
target_exp = "output_bugfix_avsr_cocktail_finetuned"  # <- anpassen

# rf-String: kombiniert raw string (\s bleibt \s) mit f-String ({target_exp} wird eingesetzt).
# re.escape() schützt vor Sonderzeichen im Verzeichnisnamen (z.B. Punkte oder Klammern).
pattern = re.compile(
    rf"Results for output dir prefix variant '{re.escape(target_exp)}':\s+"
    r"\s*Average Conversation Clustering F1 score: ([0-9.]+)\s+"
    r"\s*Average Speaker WER: ([0-9.]+)\s+"
    r"\s*Average Joint ASR-Clustering Error Rate: ([0-9.]+)",
    re.MULTILINE
)

#  search() findet den ersten (und einzigen) Treffer; None wenn nicht vorhanden
m = pattern.search(result.stdout)
if not m:
    raise ValueError(f"Kein Treffer für {target_exp} im stdout gefunden.")

# Einzeiliges DataFrame für den direkten Vergleich mit den Pre-Bugfix-Werten aus 01_
baseline_results = pd.DataFrame([{
    "exp": target_exp,
    "avg_conv_f1": float(m.group(1)),
    "avg_speaker_wer": float(m.group(2)),
    "avg_joint_error": float(m.group(3)),
}])

# Lesbareren Modellnamen erzeugen: "output_"-Prefix entfernen
baseline_results["model"] = baseline_results["exp"].str.replace("^output_", "", regex=True)
baseline_results


,exp,avg_conv_f1,avg_speaker_wer,avg_joint_error,model
0,output_bugfix_avsr_cocktail_finetuned,0.855282,0.517674,0.340167,bugfix_avsr_cocktail_finetuned


## 9 – Ergebnisse speichern

In [21]:
baseline_results.to_csv(
    "results_bugfix_baseline.csv",
    index=False
)
